In [1]:
import requests
import re
import lxml
import json
import csv
import math
import time
import progressbar
from itertools import cycle
import traceback
from bs4 import BeautifulSoup
from lxml.html import fromstring
from lxml.html.clean import Cleaner
from urllib.parse import parse_qs
import random
import pandas as pd
import googlemaps
import collections

In [2]:
json_file = open("Canadian Cities Rentals Data.json")
json_str = json_file.read()
buildings = json.loads(json_str)
data_list = []#['url', 'created', 'updated', 'email', 'name', 'address', 'city', 'province', 'country', 'postal code', 'latitude', 'longitude', 'description', 'rent/sale', 'price', 'currency', 'beds', 'baths', 'size', 'measure', 'amenities']
amenities_category_dict = {}
amenities_list = []
apartment_strings = ['Apartment', 'condo', 'corner unit']
townhouse_strings = ['Townhouse', 'townhome', 'town house', 'town home']
house_strings = ['House', 'home', 'triplex', 'duplex']
multi_unit_strings = ['multiunit', 'multi-unit','multi unit']
private_unit = ['Private Unit', 'basement unit', 'floor unit', 'bed unit', 'lower level unit', 'bachelor', 'one bedroom basement']
loft_strings = ['Loft']
bType_list = [apartment_strings, townhouse_strings, multi_unit_strings, loft_strings, private_unit, house_strings]


counter = 0
b = 0
for building in buildings:   
    url = building.get('url')
    created = building.get('created')
    updated = building.get('updated')
    email = building['contact'].get('email')            
    name = building.get('name')
    address = building['address'].get('streetAddress')
    try:
        if name in address:
            name = ""
    except:
        pass

    city = building['containedInPlace'].get('name')
    province = building['containedInPlace']['address'].get('addressRegion')
    country = building['containedInPlace']['address'].get('addressCountry')
    postal = building['address'].get('postalCode').replace(" ","")
    description = building.get('description')
    buildingType = building.get('@type')
    if 'Accommodation' in buildingType:
        for bType in bType_list:
            if any([bType_substring.lower() in description.lower() for bType_substring in bType]) is True:
                #print(url, bType[0])
                buildingType = bType[0]
                break
        if 'Accommodation' in buildingType:
            pass
            #print(url, buildingType)
    elif 'ApartmentComplex' in buildingType:
        buildingType = 'Apartment'
        
    try:
        latitude = building['geo'].get('latitude')
        longitude = building['geo'].get('longitude')
    except:
        #print(b, url)
        continue
        
    amenities = building.get('amenities')
    amen_cat_dict = {}
    amen_list = []
    for amenity in amenities:
        amen = amenity.get('name')
        amen_cat = amenity['category'].get('name')
        if amen_cat not in amenities_category_dict:
            amenities_category_dict.setdefault(amen_cat, [])                
        if amen not in amenities_category_dict[amen_cat]:
            amenities_category_dict[amen_cat].append(amen)
        if amen_cat not in amen_cat_dict:
            amen_cat_dict.setdefault(amen_cat, [])                
        if amen not in amen_cat_dict[amen_cat]:
            amen_cat_dict[amen_cat].append(amen)        
    units = building.get('containsPlace')
    i = 0
    for unit in units:
        saleType =  unit['potentialAction'].get('@type')
        price = unit['potentialAction']['priceSpecification'].get('price')
        currency = unit['potentialAction']['priceSpecification'].get('priceCurrency')
        beds = int(unit.get('name')[0])
        baths = building['units'][i]['baths']
        try:
            size = unit['floorSize'].get('value')
            measure = unit['floorSize'].get('unitCode')
        except:
            size = ""
            measure = ""
        items = {
            'Url' : url,
            'Created' : created,
            'Updated' : updated,
            'Email' : email,
            'Name' : name,
            'Address' : address,
            'City' : city,
            'Province' : province,
            'Country' : country,
            'Postal Code' : postal,
            'Latitude' : latitude,
            'Longitude' : longitude,
            'Building Type' : buildingType,
            'Description' : description,
            'Rent/Sale' : saleType,
            'Price' : price,
            'Currency' : currency,
            'Beds' : beds,
            'Baths' : baths,
            'Size' : size,
            'Measure' : measure,
            'Utilities Included' : amen_cat_dict.get('Utilities'),
            'Unit Features' : amen_cat_dict.get('Unit Features'),
            'Building Features' : amen_cat_dict.get('Building Features'),
            'Nearby Features' : amen_cat_dict.get('Nearby'),
            'Other' : amen_cat_dict.get('Other'),            
        }
        i+=1
        counter+=1
        #print(counter, name)
        data_list.append(items)
    b +=1
print(str(counter) + " Data Points Appended")
df = pd.DataFrame(data_list)


44733 Data Points Appended


In [3]:
df = pd.DataFrame(data_list)
df["Description"] = df["Description"].str.split("data:image").str[0]
df.rename(columns = {"Nearby" : "Nearby Features"}, inplace =True)

In [60]:
class RentalsCleaner:
    def splitter(self,column, string, position):
        column = column.str.split(string).str[position]
        return column
    
    def list_filter(self, column):
        clean_list = []
        for lis in column:
            if lis != None:
                for i in lis:
                    if i not in clean_list:
                        clean_list.append(i)
        clean_list.sort()
        try:
            clean_list.remove('')
        except:
            pass
        #print(clean_list)
        return clean_list
        
    def list_cleaner(self, column, clean_list):
        with open("Amenities Dictionary.json") as f:
            dictionary = json.loads(f.read())
        new_column = column
        dic = {item : "" for item in clean_list}
        u_list = []
        #print(json.dumps(dictionary, indent =2))
        for dd in dictionary:
            if new_column.name == dd:
                ddic = dictionary[dd]
                for lis in new_column:
                    if lis != None:
                        for i in lis:
                            ni = ddic.get(i, "")
                            if ni not in u_list:
                                u_list.append(ni)
            try:
                u_list.remove("")
            except:
                pass
            u_list.sort()
        #print(u_list)
        return column
        
    def item_counter(self, column, clean_list):
        dic = {item : "" for item in clean_list}
        for lis in column:
            if lis != None:
                for i in lis:
                    if i in dic:
                        dic[i] = dic[i] + 1
        dic_list = sorted(dic.items(), key=lambda x: x[1])
        for i in dic_list:
            pass
            
    def run(self):
        #df['Description'] = self.splitter(df['Description'],"data:image",0)
        
        bf_list = self.list_filter(df['Building Features'])
        df['Building Features'] = self.list_cleaner(df['Building Features'], bf_list)        
        uf_list = self.list_filter(df['Unit Features'])
        df['Unit Features'] = self.list_cleaner(df['Unit Features'], uf_list)
        nb_list = self.list_filter(df['Nearby Features'])
        df['Nearby Features'] = self.list_cleaner(df['Nearby Features'], nb_list)
        
if __name__ == "__main__":
    df = pd.DataFrame(data_list)
    cleaner = RentalsCleaner()
    cleaner.run()

In [59]:
default_dictionary = {
    "Building Features" : {
        '24 Hour Emergency Service' : 'On-Site Management',
        '24H Laundry Facility' : 'Laundry',
        '24H Maintenance' : 'On-Site Maintenance',
        '24H On-Site Maintenance' : 'On-Site Maintenance',
        '24H On-Site Security' : 'On-Site Management',
        '24H Surveillance' : 'Video Surveillance',
        '24h Security' : 'On-Site Management',
        'Air Exchanger' : 'Air Exchanger',
        'Attached Parking' : 'Parking - Covered',
        'Bbq Area/Courtyard' : 'Event Space - Courtyard',
        'Bicycle Storage' : 'Storage - Bicycle Storage',
        'Bike Racks' : 'Storage - Bicycle Storage',
        'Bike Room' : 'Storage - Bicycle Storage',
        'Billiards Room' : 'Recreation - Billiards Room',
        'Buzzer Entry' : 'Keyless Entry',
        'Card Laundry' : 'Laundry',
        'Central Air Conditioning' : 'Central Air Conditioning',
        'Central Vacuum' : 'Central Vacuum',
        'Coin Laundry' : 'Laundry',
        'Concierge' : 'On-Site Management',
        'Controlled Access' : 'Controlled Access',
        'Courtyard' : 'Event Space - Courtyard',
        'Covered Parking' : 'Parking - Covered',
        'Daycare On Site' : 'On-Site Daycare',
        'Deck' : 'Patio/Deck',
        'Detached Garage' : 'Parking - Covered',
        'Electric Vehicle Charger' : 'Parking - Electric Vehicle',
        'Elevator' : 'Elevator',
        'Elevators' : 'Elevator',
        'Energized Parking' : 'Parking - Electric Vehicle',
        'Exercise Room' : 'Fitness - Gym',
        'Fitness Area' : 'Fitness - Gym',
        'Fitness Area' : 'Fitness - Gym',
        'Fitness Centre' : 'Fitness - Gym',
        'Fitness Room' : 'Fitness - Gym',
        'Fob' : 'Keyless Entry',
        'Game Room' : 'Recreation - Game Room',
        'Garage' : 'Parking - Covered',
        'Garbage Disposal' : '',
        'Guest Suite' : '',
        'Intercom' : 'Keyless Entry',
        'Jacuzzi' : 'Jacuzzi/Hot Tub',
        'Key Card Security' : 'Keyless Entry',
        'Keyless Entry' : 'Keyless Entry',
        'Laundry Facilities' : 'Laundry',
        'Locker Room' : 'Storage - Lockers',
        'Media Room' : 'Recreation - Theatre',
        'Movie Room' : 'Recreation - Theatre',
        'Night Patrol' : 'On-Site Security',
        'On-Site Maintenance' : 'On-Site Maintenance',
        'On-Site Resident Managers' : 'On-Site Management',
        'On-site Management' : 'On-Site Management',
        'On-site Staff' : 'On-Site Management',
        'Online Payments Available' : '',
        'Outdoor Play Area' : 'Event Space - Playground',
        'Outdoor Playground' : 'Playground',
        'Outdoor Terrace' : 'Event Space - Terrace',
        'Parking - Assigned' : 'Parking',
        'Parking - Carport' : 'Parking - Covered',
        'Parking - Covered' : 'Parking - Covered',
        'Parking - Electric Vehicle' : 'Parking - Electric Vehicle',
        'Parking - Gravel' : 'Parking - Uncovered',
        'Parking - Guest' : 'Parking - Visitor',
        'Parking - Indoor' : 'Parking - Covered',
        'Parking - Inside' : 'Parking - Covered',
        'Parking - Shared' : 'Parking - Shared',
        'Parking - Street' : 'Parking - Uncovered',
        'Parking - Surfaced' : 'Parking - Uncovered',
        'Parking - Underground' : 'Parking - Covered',
        'Parking - Visitor' : 'Parking - Visitor',
        'Party Room' : 'Recreation - Party Room',
        'Patio/Deck' : 'Patio/Deck',
        'Pool - Heated' : 'Fitness - Pool',
        'Pool - Indoor' : 'Fitness - Pool',
        'Pool - Outdoor' : 'Fitness - Pool',
        'Pool - Rooftop' : 'Fitness - Pool',
        'Professionally Managed' : 'On-Site Management',
        'Recreation Room' : 'Recreation - Party Room',
        'Resident Lounge' : 'Recreation - Lounge',
        'Resident Managers' : 'On-Site Management',
        'Rooftop Deck' : 'Event Space - Terrace',
        'Rooftop Garden' : 'Event Space - Terrace',
        'Rooftop Lounge' : 'Event Space - Terrace',
        'Rooftop Pool' : 'Fitness - Pool',
        'Sauna' : 'Fitness - Sauna',
        'Scenic View' : '',
        'Secure Entry' : 'Keyless Entry',
        'Secured Access' : 'Keyless Entry',
        'Security' : 'On-Site Management',
        'Security On-site' : 'On-Site Management',
        'Security System' : 'Video Surveillance',
        'Social Room' : 'Recreation - Lounge',
        'Storage Lockers' : 'Storage - Lockers',
        'Storage Room' : 'Storage - Storage Room',
        'Sundeck' : 'Event Space - Terrace',
        'Swimming Pool' : 'Fitness - Pool',
        'Tennis Court' : 'Fitness - Tennis Courts', 
        'Tennis Courts' : 'Fitness - Tennis Courts',
        'Theatre Room' : 'Recreation - Theatre',
        'Underground Parking' : 'Parking - Covered',
        'Video Surveillance' : 'Video Surveillance',
        'Visitor Parking' : 'Parking - Visitor',
        'gym' : 'Fitness - Gym',
    },
   
    "Unit Features" : {
        'Air Conditioning' : 'Air Conditioning',
        'Alarm System' : 'Security - Alarm System',
        'Backyard' : 'Outdoor Space - Backyard',
        'Balconies' : 'Outdoor Space - Balcony',
        'Balcony' : 'Outdoor Space - Balcony',
        'Ceramic Tiles' : 'Flooring - Ceramic',
        'Dryer' : 'Laundry',
        'Ensuite Bathroom' : 'Other - Ensuite Bathroom',
        'Ensuite Laundry' : 'Laundry',
        'Fenced Yard' : 'Outdoor Space - Backyard',
        'Fireplace' : 'Other - Fireplace',
        'Flooring - Carpeted' : 'Flooring - Carpeted',
        'Flooring - Ceramic' : 'Flooring - Ceramic',
        'Flooring - Hardwood' : 'Flooring - Hardwood',
        'Flooring - Laminate' : 'Flooring - Laminate',
        'Flooring - Laminate Hardwood' : 'Flooring - Laminate',
        'Flooring - Laminate Wide Plank' : 'Flooring - Laminate',
        'Flooring - Mixed Options' : 'Flooring - Mixed',
        'Flooring - Parquet' : 'Flooring - Parquet',
        'Flooring - Tile' : 'Flooring - Tile',
        'Flooring - Vinyl' : 'Flooring - Vinyl',
        'Flooring - Vinyl Plank' : 'Flooring - Vinyl',
        'Flooring - Vinyl/Lino' : 'Flooring - Vinyl',
        'Flooring - Wood' : 'Flooring - Hardwood',
        'Freezer' : 'Appliances - Freezer',
        'Freshly Painted' : 'Status - Freshly Painted',
        'Fridge' : 'Appliances - Refrigerator',
        'Fully Equipped Kitchen' : 'Status - Fully Equipped Kitchen',
        'Furnished' : 'Status - Furnished',
        'Garburator' : 'Appliances - Garburator',
        'Garden' : 'Other - Garden',
        'Gas Heating' : 'Heating - Gas',
        'Granite Countertops' : 'Finishes - Granite Countertops',
        'Harwood Floors' : 'Flooring - Hardwood',
        'Hood Fan' : 'Appliances - Hood Fan',
        'Hot Tub' : 'Other - Jacuzzi/Hot Tub',
        'In-Suite Laundry' : 'Laundry',
        'Individual Thermostats' : 'Heating - Individual Thermostats',
        'Internet Ready' : 'Status - Internet Ready',
        'Island' : 'Design - Island',
        'Large Windows' : 'Design - Large Windows',
        'Led Lighting' : 'Lighting - LED',
        'Microwave' : 'Appliances - Microwave',
        'New Appliances' : 'Status - New Appliances',
        'Newly Renovated' : 'Status - Newly Renovatd',
        'Open Concept Design' : 'Design - Open Concept',
        'Oven' : 'Appliances - Oven',
        'Parking' : 'Parking',
        'Patio' : 'Outdoor Space - Patio/Deck',
        'Pool - Private' : 'Other - Pool',
        'Private Entry' : 'Security - Private Entry',
        'Private Yard' : 'Outdoor Space - Backyard',
        'Radiant Heat' : 'Heating - Radiant',
        'Refrigerator' : 'Appliances - Refrigerator',
        'Security Cameras' : 'Security - Security Cameras',
        'Shared Yard' : 'Outdoor Space - Backyard',
        'Stainless Steel Appliances' : 'Finishes - Stainless Steel Appliances',
        'Storage' : 'Other - Storage',
        'Stove' : 'Kitchen Appliances - Stove',
        'Terrace' : 'Outdoor Space - Rooftop/Terrace',
        'Utilities Included' : 'Utilities Included',
        'Walk-Out Patio' : 'Outdoor Space - Patio/Deck',
        'Walk-in Closet' : 'Other - Walk-in Closet',
        'Wall Unit' : 'Air Conditioning - Wall Unit',
        'Washer' : 'Laundry',
        'Washer/Dryer' : 'Laundry',
        'Wheelchair Access' : 'Other - Wheelchair Access',
        'Window Air Conditioner' : 'Air Conditioning - Wall Unit',
        'Wine Fridge' : 'Other - Wine Fridge',
    },
    
    "Nearby Features" : {
        '24h Emergency' : 'Service - Hospital', 
        'Airport' : 'Service - Airport',
        'Atm' : 'Service - Bank',
        'Bank' : 'Service - Bank',
        'Bars' : 'Entertainment - Bars/Clubs',
        'Basketball Court' : 'Fitness - Park/Playground',
        'Bike Trail' : 'Fitness - Bike Trails',
        'Bike Trails' : 'Fitness - Bike Trails',
        'Bus Routes' : 'Transportation - Bus',
        'Bus Stop' : 'Transporations - Bus',
        'Cafe' : 'Service - Cafe',
        'Canadian Superstore' : 'Service - Superstore',
        'Canadian Tire' : 'Service - Superstore',
        'Car Wash' : 'Service - Gas Station',
        'Church' : 'Other - Church',
        'Clinics' : 'Service - Hospital',
        'Club House' : 'Entertainment - Club House',
        'Community Garden' : 'Fitness - Park/Playground',
        'Convenience Store' : 'Service - Convenience Store',
        'Costco' : 'Service - Superstore',
        'Daycare' : 'Service - Daycare',
        'Dog Park' : 'Fitness - Park/Playground',
        'Gas Station' : 'Service - Gas Station',
        'Golf Course' : 'Entertainment - Golf Course',
        'Grocery Store' : 'Service - Grocery Store',
        'Gym' : 'Fitness - Gym',
        'Highway' : 'Tranportation - Highway',
        'Home Depot' : 'Service - Home Depot',
        'Hospital' : 'Service - Hospital',
        'Lake Trails' : 'Fitness - Trails',
        'Lrt' : 'Transportation - LRT',
        'Marina' : 'Other - Marina',
        'Mcdonalds' : 'Service - Restaurant',
        'Movie Theater' : 'Entertainment - Movie Theatre',
        'No Frills' : 'Service - Supermarket',
        'Parks' : 'Fitness - Park/Playground',
        'Playground' : 'Fitness - Park/Playground',
        'Playground/ Park' : 'Fitness - Park/Playground',
        'Pool' : 'Fitness - Pool',
        'Public Library' : 'Education - Library',
        'Public Transit' : 'Transportation - Public Transit',
        'Recreation' : 'Entertainment - Recreation',
        'Restaurants' : 'Service - Restaurant',
        'River' : 'Fitness - River',
        'Running Path' : 'Fitness - Running Trails',
        'School' : 'Education - School',
        'Schools' : 'Education - School',
        'Shopping' : 'Entertainment - Shopping Centre',
        'Shopping Centre' : 'Entertainment - Shopping Centre',
        'Sobeys' : 'Service - Supermarket',
        'Sports Complex' : 'Fitness - Gym',
        'Tim Hortons' : 'Service - Cafe',
        'University' : 'Education - University',
        'Walmart' : 'Service - Superstore', 
    },
}

with open('Amenities Dictionary.json', 'w') as fp:
    json.dump(default_dictionary, fp)
   